In [1]:
# Configure sys.path so repo modules are importable
import sys, importlib
from pathlib import Path
REPO_ROOT = Path.cwd().resolve()
while REPO_ROOT != REPO_ROOT.parent and not ((REPO_ROOT / 'src').exists() and (REPO_ROOT / 'ArtWork').exists()):
    REPO_ROOT = REPO_ROOT.parent
if not (REPO_ROOT / 'src').exists():
    raise RuntimeError('Run this notebook inside the XMODE repository.')
ARTWORK_DIR = REPO_ROOT / 'ArtWork'
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
if str(ARTWORK_DIR) not in sys.path:
    sys.path.insert(1, str(ARTWORK_DIR))
sys.modules['tools'] = importlib.import_module('ArtWork.tools')
sys.modules['tools.backend'] = importlib.import_module('ArtWork.tools.backend')


In [2]:
# Create a tiny SQLite DB referencing nature.jpg
import sqlite3
from pathlib import Path
TEST_DIR = Path('bhy_test')
TEST_DIR.mkdir(exist_ok=True)
IMAGE_PATH = TEST_DIR / 'nature.jpg' if (TEST_DIR / 'nature.jpg').exists() else Path('nature.jpg')
if not IMAGE_PATH.exists():
    raise FileNotFoundError('Place nature.jpg in bhy_test/ or repo root.')
DB_PATH = TEST_DIR / 'qwen_demo.db'
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS paintings')
cur.execute('CREATE TABLE paintings (title TEXT, inception TEXT, movement TEXT, genre TEXT, image_url TEXT, img_path TEXT)')
cur.execute('INSERT INTO paintings VALUES (?, ?, ?, ?, ?, ?)',
            ("Demo Nature Scene", '1500-01-01 00:00:00', 'Nature Revival', 'Landscape', 'local://nature', IMAGE_PATH.as_posix()))
conn.commit(); conn.close()
print('Test DB ->', DB_PATH)
print('Image    ->', IMAGE_PATH)


Test DB -> bhy_test/qwen_demo.db
Image    -> nature.jpg


In [3]:
# Point Artwork settings to the test assets
import os, getpass
from src.settings import get_settings
settings = get_settings()
ARTWORK_CFG = settings.artwork
ARTWORK_CFG.db_path = DB_PATH
ARTWORK_CFG.questions_file = TEST_DIR / 'questions_nature.json'
ARTWORK_CFG.questions_file.write_text('''[
  "Describe the Demo Nature Scene."
]''', encoding='utf-8')
ARTWORK_CFG.log_dir = TEST_DIR / 'logs'
ARTWORK_CFG.log_dir.mkdir(exist_ok=True)
ARTWORK_CFG.thread_id = 'bhy-qwen-demo'

def _need(var):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_need('OPENAI_API_KEY')
if settings.models.provider.lower() == 'anthropic':
    _need('ANTHROPIC_API_KEY')
print('Artwork DB ->', ARTWORK_CFG.db_path)
print('VQA provider ->', settings.artwork.vqa_provider)


Artwork DB -> bhy_test/qwen_demo.db
VQA provider -> blip


In [4]:
# Direct call to Qwen3-VL via Ollama
from ArtWork.tools.backend.qwen_ollama import VisualQAOllama
ollama_vqa = VisualQAOllama(model=settings.artwork.ollama_model,
                            endpoint=settings.artwork.ollama_endpoint)
question = 'What do you observe in this nature painting?'
answers = ollama_vqa.extract([IMAGE_PATH.as_posix()], question)
print('Ollama response:', answers[0])


ReadTimeout: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=60)

In [ ]:
# Build the LangGraph chain (Claude + configured VQA backend)
from ArtWork.src.build_graph import graph_construction
provider = settings.models.provider.lower()
model = settings.models.default_chat_model if provider == 'openai' else settings.models.anthropic_model
temperature = settings.models.default_temperature
chain = graph_construction(model=model,
                           temperature=temperature,
                           db_path=str(ARTWORK_CFG.db_path),
                           log_path=str(ARTWORK_CFG.log_dir),
                           vqa_provider=settings.artwork.vqa_provider,
                           settings=settings)
print('Graph ready -> LLM:', settings.models.provider, '| VQA:', settings.artwork.vqa_provider)


In [ ]:
# Ask Claude to describe the demo painting
question = 'How many trees are in the picture.'
config = {'configurable': {'thread_id': 'bhy-qwen-demo'}}
print('Question:', question)
for step in chain.stream(question, config, stream_mode='values'):
    for message in step:
        print(message)
    print('---')
